# Covid-19 RKI Zahlen

Fokus auf die Zahlen nach der letzten Anpassung der Teststrategie im November.
Analyse verschiedener vom RKI ausgegebenen Dokumente.

## Relevante Links:
https://www.zdf.de/nachrichten/heute/coronavirus-ausbreitung-infografiken-102.html

In [ ]:
!pip install jsonpath2

In [ ]:
import pandas as pd

# https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Daten/Testzahlen-gesamt.xlsx;__blob=publicationFile
## data is updated every wednesday
df = pd.read_excel(
    '../input/20201202-rki-testzahlen/Testzahlen-gesamt.xlsx', 
    'Testzahlen', 
    header=2,
    usecols='B:D,F',
    # skiprows=1,
    skipfooter=1
)

# clean up
df = df.drop(38, axis=0)

df.rename(columns = {"Kalenderwoche 2020": "KW"},  
          inplace = True) 

df.loc[36, 'KW'] = 46
df.loc[37, 'KW'] = 47

df.loc[:, 'Positivquote'] = df.loc[:, 'Positiv getestet'] / df.loc[:, 'Anzahl Testungen']
df.loc[:, 'Vorwoche'] = df.loc[:, 'Positiv getestet'].shift(1)
df.loc[:, 'Vorwoche delta'] = df.loc[:, 'Positiv getestet'].diff(1)
df_testzahlen = df.copy()

df = pd.read_excel(
    '../input/20201202-rki-testzahlen/Testzahlen-gesamt.xlsx', 
    'Probenrückstau'
)
df_rückstau = df.copy()


df = pd.read_excel(
    '../input/20201202-rki-testzahlen/Testzahlen-gesamt.xlsx', 
    'Testkapazitäten',
    header=1,
    usecols='A:B,E'
)
df.rename(columns = {"KW, für die die Angabe prognostisch erfolgt ist:": "KW"},  
          inplace = True) 
df.loc[:, 'KW'] = df.loc[:, 'KW'].apply(lambda str: int(str[2:4]))
df_kapazitäten = df.copy()

df = df.merge(df_testzahlen, on='KW') \
    .merge(df_rückstau, on='KW') \
    .set_index('KW')

# display(df.head())
display(df.tail())

# Übersicht Testkapazitäten und Positivquote

In [ ]:
import matplotlib.pyplot as plt

start_index = 33

plt.plot(df.loc[start_index:, 'Positivquote'], label='Positivquote')
plt.legend()
plt.show()

plt.plot(df.loc[start_index:, 'Positiv getestet'], label='Positiv getestet')
plt.plot(df.loc[start_index:, 'Probenrückstau'], label='Rückstau')
plt.plot((df.loc[start_index:, 'Positiv getestet'] + df.loc[start_index:, 'Probenrückstau'] * df.loc[start_index:, 'Positivquote']), label='Projektion Rückstau')
plt.legend()
plt.show()

* um KW 35 entstand ein Rückstau, während die allgemeine Positivquote lag bei unter 2%
* um KW 44 entstand ein Rückstau, während die allgemeine Positivquote lag bei ca. 7%

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df.loc[start_index:, 'Reale Testkapazität zum Zeitpunkt der Abfrage'], label='Testkapazitäten')
plt.plot(df.loc[start_index:, 'Reale Testkapazität zum Zeitpunkt der Abfrage'] + df.loc[start_index:, 'Probenrückstau'], label='Abstriche')

plt.legend()
ax = plt.gca()
# plt.plot([0, 1], [0, 1], transform=ax.transAxes, alpha=0.3)
plt.ticklabel_format(scilimits=(0,10), style = 'sci')
plt.show()

Gemäß angepasster Teststrategie ab 3.11.2020, KW 46

In [ ]:
import pandas as pd

def data_loader(): return load_from_arcgis_hub()

def load_from_tracking():
    df_tracking = pd.read_csv(
        '../input/covid19-tracking-germany/covid_de.csv',
        usecols = ['date', 'cases', 'deaths', 'recovered'],
        parse_dates = ['date']
    )
    return df_tracking

def load_from_rki():
    df_rki = pd.read_csv(
        '../input/covid19-data-germany-robert-koch-institute/dd4580c810204019a7b8eb3e0b329dd6_0.csv',
        usecols = ['Refdatum', 'Meldedatum', 'AnzahlFall', 'AnzahlTodesfall', 'AnzahlGenesen'],
        parse_dates = ['Refdatum', 'Meldedatum']
    )
    df_rki.rename(
        columns={'Meldedatum':'date', 'AnzahlFall':'cases', 'AnzahlTodesfall':'deaths', 'AnzahlGenesen':'recovered'}, 
        inplace=True
    )
    return df_rki

def load_from_arcgis_hub():
    from datetime import date
    filename = date.today().strftime('arcgis_hub_rki_covid_%Y%m%d.csv')
    
    from os.path import exists
    if exists(filename):
        df_rki_cached = pd.read_csv(filename)
        return df_rki_cached
    
    from urllib.request import urlopen
    import json
    from jsonpath2 import match
    
    api_data = []
    resultOffset = 0
    exceededTransferLimit = True
    
    while exceededTransferLimit:
        print('offset: %d' % resultOffset)
        res = urlopen('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/RKI_COVID19/FeatureServer/0/query' + 
                      '?where=1%3D1&f=json'+ 
                      '&outFields=AnzahlFall,AnzahlTodesfall,Refdatum,AnzahlGenesen,Meldedatum' +
                      '&resultOffset=' + str(resultOffset)
        )
        raw_json = res.read()
        parsed_json = json.loads(raw_json)
        
        try:
            exceededTransferLimit = parsed_json['exceededTransferLimit']
        except KeyError:
            exceededTransferLimit = False
        
        matches = match('$.features.*.attributes', parsed_json)

        for m in matches:
            cv = m.current_value
            try:
                api_data.append(
                    {
                        'date': date.fromtimestamp(cv['Meldedatum']/1000), 
                        'cases': cv['AnzahlFall'], 
                        'deaths': cv['AnzahlTodesfall'], 
                        'recovered': cv['AnzahlGenesen']
                    }
                )
            except ValueError:
                import time
                print("bad data: %s %s" % (cv, time.time()))
                
            resultOffset = len(api_data)
    
    df_rki_download = pd.DataFrame(api_data)
    df_rki_download.to_csv(filename)
    return df_rki_download

df = data_loader()
    
display(df.head())
display(df.tail())

In [ ]:
df_agg = df.groupby(by=['date'], as_index=True).agg(
    {'cases': sum, 'deaths': sum, 'recovered': sum}
).sort_values('date')

startindex = pd.to_datetime('2020-10-01')
ticks = pd.date_range(startindex, df_agg.index[-1], freq='W')

display(df_agg.tail())

In [ ]:
# https://stackoverflow.com/questions/45281297/group-by-week-in-pandas

df_rki_kumm = pd.read_excel(
    '../input/20201211-rki-fallzahlen-kummuliert/Fallzahlen_Kum_Tab.xlsx', 
    'Fälle-Todesfälle-gesamt',
    header=2,
    usecols='A:B,D:F',
    index_col=0,
    # parse_dates=False
)

def parseDate(sth):
    from datetime import datetime
    if sth == '25.08,2020':
        return datetime(2020, 8, 25)
    elif isinstance(sth, str):
        return datetime.strptime(sth, '%d.%m.%Y')
    elif isinstance(sth, datetime):
        return datetime(sth.year, sth.month, sth.day)
    else:
        return sth
    
df_rki_kumm.index = df_rki_kumm.index.map(parseDate)

display(df_agg.loc[:, 'deaths'].cumsum().tail())
display(df_rki_kumm.loc[:, 'Differenz Vortag Todesfälle'].tail())

# Übersicht Fallzahlen und Todesfälle

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks

peakX_cases = find_peaks(df_agg.loc[startindex:, 'cases'].values)[0]
peakX_recovered = find_peaks(df_agg.loc[startindex:, 'recovered'].values)[0]
peakX_deaths = find_peaks(df_rki_kumm.loc[startindex:, 'Differenz Vortag Todesfälle'].values)[0]

fig, ax = plt.subplots(figsize=(20,10))
ax.plot(df_agg.loc[startindex:, 'cases'], color='orange', alpha=0.5, label='Infiziert')
ax.plot(df_agg.loc[startindex:, 'cases'].iloc[peakX_cases], color='darkorange', linewidth=3)

ax.plot(df_agg.loc[startindex:, 'recovered'], color='green', alpha=0.5, label='Genesen')
ax.plot(df_agg.loc[startindex:, 'recovered'].iloc[peakX_recovered], color='darkgreen', linewidth=3, label='Genesen')

ax.legend()
ax.grid()

ax = ax.twinx()
ax.plot(df_rki_kumm.loc[startindex:, 'Differenz Vortag Todesfälle'], color='red', alpha=0.5, label='Todesfälle')
ax.plot(df_rki_kumm.loc[startindex:, 'Differenz Vortag Todesfälle'].iloc[peakX_deaths], color='darkred', linewidth=3)

ax.set_ylabel('Todesfälle (7T)', color='red')
# ax.legend()

plt.xticks(ticks)

plt.show()

# Changelog
* initiale Version / Datenstand 26.11.2020
* update Datenstand 03.12.2020
* adopted currated dataset provided on kaggle
* update Datenstand 09.12.2020, bessere Darstellung
* Refdatum für Meldedatum getauscht
* Todesfälle aus alternativer Quelle des RKI bezogen
* Daten per API ziehen, Datenstand 17.12.2020